<div style="float:right; padding-top: 15px; padding-right: 15px">
    <div>
        <a href="https://whiteboxml.com">
            <img src="https://whiteboxml.com/static/img/logo/black_bg_white.svg" width="250">
        </a>
    </div>
</div>

# action queries

## 1. introduction

* action queries **modify databases**
* action queries are **not to retrieve information**, but to :
    * write new info
    * modify existing information
    * modify database structure (tables, etc.)

## 2. sample database (publications 📘)

In [1]:
# let's load jupyter sql extension

%load_ext sql
%config SqlMagic.autocommit = False

In [2]:
# load database

%sql sqlite:///data/publications.db

getting tables in publications database:

In [3]:
%%sql tables <<

SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

 * sqlite:///data/publications.db
Done.
Returning data to local variable tables


In [4]:
tables.DataFrame()

,name
0,authors
1,discounts
2,employee
3,jobs
4,pub_info
5,publishers
6,roysched
7,sales
8,stores
9,titleauthor


## 3. types of action queries

* create / delete / modify tables
* appending / deleting / updating records in a table

### table creation

![sqlite create table](https://www.sqlite.org/images/syntax/create-table-stmt.gif "SQLite CREATE TABLE")

### a) create table (as is, not as select)

```sql
CREATE TABLE [IF NOT EXISTS] [schema_name].table_name (
    column_1 data_type PRIMARY KEY,
       column_2 data_type NOT NULL,
    column_3 data_type DEFAULT 0,
    table_constraints
) [WITHOUT ROWID];
```

Let's create an empty table with:
* a primary key called **my_pk_col** which is as integer
* a string column called **one_column**
* a datetime column called **another_column**

In [5]:
%%sql create <<

CREATE TABLE IF NOT EXISTS my_custom_table(
    my_pk_col integer primary key,
    one_column string,
    another_column datetime default '1989-07-21 01:00:00'
)

 * sqlite:///data/publications.db
Done.
Returning data to local variable create


In [6]:
create.DataFrame()  # no output for action queries!

""


let's check tables in database again!

In [7]:
%%sql tables <<

SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

 * sqlite:///data/publications.db
Done.
Returning data to local variable tables


In [8]:
tables.DataFrame()

,name
0,authors
1,discounts
2,employee
3,jobs
4,pub_info
5,publishers
6,roysched
7,sales
8,stores
9,titleauthor


see **my custom table**? use DBeaver to check its content...

### b) create table (as select)

now, let's create a new table with the result of a query... first let's make a query to retrieve the number of distinct orders, titles for every book store, as well as total sales...

In [9]:
%%sql select <<

select
stores.stor_id, 
stores.stor_name, 
count(distinct sales.ord_num),
count(distinct sales.title_id),
sum(sales.qty)
from sales join stores on
stores.stor_id = sales.stor_id
group by stores.stor_id, stores.stor_name;

 * sqlite:///data/publications.db
Done.
Returning data to local variable select


In [10]:
select.DataFrame()

,stor_id,stor_name,count(distinct sales.ord_num),count(distinct sales.title_id),sum(sales.qty)
0,6380,Eric the Read Books,2,2,8
1,7066,Barnum's,2,2,125
2,7067,News & Brews,2,4,90
3,7131,Doc-U-Mat: Quality Laundry and Books,3,6,130
4,7896,Fricative Bookshop,3,3,60
5,8042,Bookbeat,4,4,80


finally, let's create a table from this query: 

In [11]:
%%sql create_as_select <<

create table if not exists store_sales_summary 
as 
select
stores.stor_id, 
stores.stor_name, 
count(distinct sales.ord_num),
count(distinct sales.title_id),
sum(sales.qty)
from sales join stores on
stores.stor_id = sales.stor_id
group by stores.stor_id, stores.stor_name;

 * sqlite:///data/publications.db
Done.
Returning data to local variable create_as_select


In [12]:
create_as_select.DataFrame()  # no output for action queries!

""


new table should be in the database. this one will have the columns and rows from specified query:

In [13]:
%%sql select <<

select * from store_sales_summary;

 * sqlite:///data/publications.db
Done.
Returning data to local variable select


In [14]:
select.DataFrame()

,stor_id,stor_name,count(distinct sales.ord_num),count(distinct sales.title_id),sum(sales.qty)
0,6380,Eric the Read Books,2,2,8
1,7066,Barnum's,2,2,125
2,7067,News & Brews,2,4,90
3,7131,Doc-U-Mat: Quality Laundry and Books,3,6,130
4,7896,Fricative Bookshop,3,3,60
5,8042,Bookbeat,4,4,80


### drop table (be careful! 🚨)

![sqlite drop table](https://www.sqlite.org/images/syntax/drop-table-stmt.gif "SQLite DROP TABLE")

be very careful, specially if you do not have a backup... how to make a backup? (production server example)... let's drop **my_custom_table**...

In [15]:
%%sql drop <<

drop table if exists my_custom_table;

 * sqlite:///data/publications.db
Done.
Returning data to local variable drop


In [16]:
drop.DataFrame()

""


now the table should not be in the database anymore...

In [17]:
%%sql tables <<

SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

 * sqlite:///data/publications.db
Done.
Returning data to local variable tables


In [18]:
tables.DataFrame()

,name
0,authors
1,discounts
2,employee
3,jobs
4,pub_info
5,publishers
6,roysched
7,sales
8,stores
9,titleauthor


perfect!

### alter table

![sqlite alter table](https://www.sqlite.org/images/syntax/alter-table-stmt.gif "SQLite ALTER TABLE")

### a) add new column to table

let's add a new column to this table with the sales goal for every store... will set a default of 100 units and the column must not be nullable...

In [19]:
%%sql alter_table_add_col <<

ALTER TABLE store_sales_summary 
ADD COLUMN goal INTEGER DEFAULT 100 NOT NULL;

 * sqlite:///data/publications.db
Done.
Returning data to local variable alter_table_add_col


In [20]:
alter_table_add_col.DataFrame()

""


In [21]:
%%sql select <<

select * from store_sales_summary;

 * sqlite:///data/publications.db
Done.
Returning data to local variable select


In [22]:
select.DataFrame()

,stor_id,stor_name,count(distinct sales.ord_num),count(distinct sales.title_id),sum(sales.qty),goal
0,6380,Eric the Read Books,2,2,8,100
1,7066,Barnum's,2,2,125,100
2,7067,News & Brews,2,4,90,100
3,7131,Doc-U-Mat: Quality Laundry and Books,3,6,130,100
4,7896,Fricative Bookshop,3,3,60,100
5,8042,Bookbeat,4,4,80,100


### b) rename table

we have create a sales benchmarking table with a goal for every store... let's rename our table to reflect this...

In [23]:
%%sql alter_table_rename_table <<

ALTER TABLE store_sales_summary RENAME TO sales_benchmarking;

 * sqlite:///data/publications.db
Done.
Returning data to local variable alter_table_rename_table


In [24]:
alter_table_rename_table.DataFrame()

""


let's check table names in database...

In [25]:
%%sql tables <<

SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

 * sqlite:///data/publications.db
Done.
Returning data to local variable tables


In [26]:
tables.DataFrame()

,name
0,authors
1,discounts
2,employee
3,jobs
4,pub_info
5,publishers
6,roysched
7,sales
8,stores
9,titleauthor


### c) alter table drop column

not available for SQLite 😓, maybe for Postgres or MySQL...

### d) alter table rename column

let's rename those ugly column names!

In [27]:
%%sql alter_table_rename_col <<

ALTER TABLE  sales_benchmarking RENAME COLUMN  
"count(distinct sales.ord_num)" TO total_orders;

 * sqlite:///data/publications.db
Done.
Returning data to local variable alter_table_rename_col


In [28]:
%%sql alter_table_rename_col <<

ALTER TABLE  sales_benchmarking RENAME COLUMN  
"count(distinct sales.title_id)" TO unique_titles;

 * sqlite:///data/publications.db
Done.
Returning data to local variable alter_table_rename_col


In [29]:
%%sql alter_table_rename_col <<

ALTER TABLE  sales_benchmarking RENAME COLUMN  
"sum(sales.qty)" TO total_qty;

 * sqlite:///data/publications.db
Done.
Returning data to local variable alter_table_rename_col


In [30]:
%%sql select <<

select * from sales_benchmarking;

 * sqlite:///data/publications.db
Done.
Returning data to local variable select


In [31]:
select

stor_id,stor_name,total_orders,unique_titles,total_qty,goal
6380,Eric the Read Books,2,2,8,100
7066,Barnum's,2,2,125,100
7067,News & Brews,2,4,90,100
7131,Doc-U-Mat: Quality Laundry and Books,3,6,130,100
7896,Fricative Bookshop,3,3,60,100
8042,Bookbeat,4,4,80,100


### delete records in a table

![sqlite delete](https://www.sqlite.org/images/syntax/delete-stmt.gif "SQLite DELETE")

let's drop poor performance stores with less than 90 units sold...

In [32]:
%%sql drop_records <<

DELETE FROM sales_benchmarking
WHERE total_qty < 90;

 * sqlite:///data/publications.db
3 rows affected.
Returning data to local variable drop_records


In [33]:
drop_records.DataFrame()

""


check results...

In [35]:
%%sql select <<

select * from sales_benchmarking;

 * sqlite:///data/publications.db
Done.
Returning data to local variable select


In [36]:
select

stor_id,stor_name,total_orders,unique_titles,total_qty,goal
7066,Barnum's,2,2,125,100
7067,News & Brews,2,4,90,100
7131,Doc-U-Mat: Quality Laundry and Books,3,6,130,100


### insert new rows

![sqlite insert](https://www.sqlite.org/images/syntax/insert-stmt.gif "SQLite INSERT")

### a) append (insert) queries, as is (not as select)

let's append a top performer fnac store...look at the default value after executing this query... :-D

In [37]:
%%sql insert_into <<

INSERT INTO sales_benchmarking 
(stor_id, 
 stor_name, 
 total_orders,
 unique_titles,
 total_qty)
VALUES (1, "fnac_callao", 1000, 1000, 1000);

 * sqlite:///data/publications.db
1 rows affected.
Returning data to local variable insert_into


In [141]:
insert_into.DataFrame()

""


check results...

In [38]:
%%sql select <<

select * from sales_benchmarking;

 * sqlite:///data/publications.db
Done.
Returning data to local variable select


In [39]:
select

stor_id,stor_name,total_orders,unique_titles,total_qty,goal
7066,Barnum's,2,2,125,100
7067,News & Brews,2,4,90,100
7131,Doc-U-Mat: Quality Laundry and Books,3,6,130,100
1,fnac_callao,1000,1000,1000,100


### b) append (insert) queries as select

let's recover our poor performers...

be careful if modifications have been made to the table... rows must be equivalent, run the cell to drop and create store_sales_summary before this one...

In [40]:
%%sql insert_into_as_select <<

insert into sales_benchmarking
select 
stores.stor_id, 
stores.stor_name, 
count(distinct sales.ord_num) as total_orders,
count(distinct sales.title_id) as unique_titles,
sum(sales.qty) as total_qty,
100
from sales join stores on
stores.stor_id = sales.stor_id
group by stores.stor_id, stores.stor_name
having sum(sales.qty) < 90;

 * sqlite:///data/publications.db
3 rows affected.
Returning data to local variable insert_into_as_select


In [145]:
insert_into_as_select.DataFrame()

""


check results...

In [41]:
%%sql select <<

select * from sales_benchmarking;

 * sqlite:///data/publications.db
Done.
Returning data to local variable select


In [42]:
select

stor_id,stor_name,total_orders,unique_titles,total_qty,goal
7066,Barnum's,2,2,125,100
7067,News & Brews,2,4,90,100
7131,Doc-U-Mat: Quality Laundry and Books,3,6,130,100
1,fnac_callao,1000,1000,1000,100
6380,Eric the Read Books,2,2,8,100
7896,Fricative Bookshop,3,3,60,100
8042,Bookbeat,4,4,80,100


### update

![sqlite insert](https://www.sqlite.org/images/syntax/update-stmt.gif "SQLite INSERT")

### a) update all rows

let's increate the goal for all our stores...

In [43]:
%%sql update <<

update sales_benchmarking
set goal = goal + 50;

 * sqlite:///data/publications.db
7 rows affected.
Returning data to local variable update


In [44]:
update.DataFrame()

""


check results...

In [45]:
%%sql select <<

select * from sales_benchmarking;

 * sqlite:///data/publications.db
Done.
Returning data to local variable select


In [46]:
select

stor_id,stor_name,total_orders,unique_titles,total_qty,goal
7066,Barnum's,2,2,125,150
7067,News & Brews,2,4,90,150
7131,Doc-U-Mat: Quality Laundry and Books,3,6,130,150
1,fnac_callao,1000,1000,1000,150
6380,Eric the Read Books,2,2,8,150
7896,Fricative Bookshop,3,3,60,150
8042,Bookbeat,4,4,80,150


### update rows conditionally

let's increate goal in +50 units only for those stores with less than 4 total orders...

In [47]:
%%sql update_conditional <<

update sales_benchmarking
set goal = goal + 50
where total_orders < 4;

 * sqlite:///data/publications.db
5 rows affected.
Returning data to local variable update_conditional


In [48]:
update_conditional.DataFrame()

""


check results...

In [49]:
%%sql select <<

select * from sales_benchmarking;

 * sqlite:///data/publications.db
Done.
Returning data to local variable select


In [50]:
select

stor_id,stor_name,total_orders,unique_titles,total_qty,goal
7066,Barnum's,2,2,125,200
7067,News & Brews,2,4,90,200
7131,Doc-U-Mat: Quality Laundry and Books,3,6,130,200
1,fnac_callao,1000,1000,1000,150
6380,Eric the Read Books,2,2,8,200
7896,Fricative Bookshop,3,3,60,200
8042,Bookbeat,4,4,80,150


## 4. bonus: operate with sqlalchemy over databases

In [51]:
from sqlalchemy import create_engine

In [52]:
eng = create_engine('sqlite:///data/publications.db')

In [53]:
eng.execute("select * from sales;").fetchone()

('6380', '6871', '1994-09-14 00:00:00', 5, 'Net 60', 'BU1032')

https://docs.sqlalchemy.org/en/13/core/connections.html

## 5. bonus: case when

### select

https://www.sqlitetutorial.net/sqlite-case/

In [54]:
%%sql case_when_select <<

select 
    case
        when goal >= 200 
        then 'too much'
        else 'ok'
    end
from sales_benchmarking;

 * sqlite:///data/publications.db
Done.
Returning data to local variable case_when_select


In [55]:
case_when_select.DataFrame()

,case\n when goal >= 200 \n then 'too much'\n else 'ok'\n end
0,too much
1,too much
2,too much
3,ok
4,too much
5,too much
6,ok


### update

In [56]:
%%sql case_when_update <<

update sales_benchmarking
set goal =
    case
        when goal >= 200 
        then 'too much'
        else 'ok'
    end;

 * sqlite:///data/publications.db
7 rows affected.
Returning data to local variable case_when_update


In [57]:
case_when_update.DataFrame()

""


check results...

In [59]:
%%sql select <<

select * from sales_benchmarking;

 * sqlite:///data/publications.db
Done.
Returning data to local variable select


In [60]:
select

stor_id,stor_name,total_orders,unique_titles,total_qty,goal
7066,Barnum's,2,2,125,too much
7067,News & Brews,2,4,90,too much
7131,Doc-U-Mat: Quality Laundry and Books,3,6,130,too much
1,fnac_callao,1000,1000,1000,ok
6380,Eric the Read Books,2,2,8,too much
7896,Fricative Bookshop,3,3,60,too much
8042,Bookbeat,4,4,80,ok


## 6. bonus: transactional vs analytical databases

https://techdifferences.com/difference-between-oltp-and-olap.html

<div style="padding-top: 25px; float: right">
    <div>    
        <i>&nbsp;&nbsp;© Copyright by</i>
    </div>
    <div>
        <a href="https://whiteboxml.com">
            <img src="https://whiteboxml.com/static/img/logo/black_bg_white.svg" width="125">
        </a>
    </div>
</div>